In [ ]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, mean_squared_error

import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv('data/undersampling/mammography_train.csv')
test = pd.read_csv('data/undersampling/mammography_test.csv')
val = pd.read_csv('data/undersampling/mammography_validation.csv')

X_train = train.iloc[:, :-1].values
X_test = test.iloc[:, :-1].values
X_val = val.iloc[:, :-1].values

y_train = train.iloc[:, -1].values
y_test = test.iloc[:, -1].values
y_val = val.iloc[:, -1].values

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [ ]:
learning_rate = 0.0005
hidden_layers = 5
act_function = 'relu'

neurons = 16
decay = 0

In [ ]:
def create_network():
    # Aqui criamos o esboço da rede.
    classifier = Sequential()

    # Agora adicionamos a primeira camada escondida contendo 16 neurônios e função de ativação
    # tangente hiperbólica. Por ser a primeira camada adicionada à rede, precisamos especificar
    # a dimensão de entrada (número de features do data set), que no caso do mammography são 6.
    classifier.add(Dense(neurons, activation=act_function, input_dim=6))
    
    for i in range(hidden_layers-1):
        classifier.add(Dense(neurons, activation=act_function))

    # Em seguida adicionamos a camada de saída. Como nosso problema é binário só precisamos de
    # 1 neurônio com função de ativação sigmoidal. A partir da segunda camada adicionada keras já
    # consegue inferir o número de neurônios de entrada (16) e nós não precisamos mais especificar.
    classifier.add(Dense(1, activation='sigmoid'))

    # Por fim compilamos o modelo especificando um otimizador, a função de custo, e opcionalmente
    # métricas para serem observadas durante treinamento.
    adam = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=decay)
    classifier.compile(optimizer=adam, loss='mean_squared_error')
    return classifier

In [ ]:
def train_network(classifier):
    return classifier.fit(X_train, y_train, batch_size=64, epochs=10000, 
                         callbacks=[EarlyStopping(patience=10)],
                         validation_data=(X_val, y_val))

In [ ]:
def extract_final_losses(history):
    """Função para extrair o melhor loss de treino e validação.
    
    Argumento(s):
    history -- Objeto retornado pela função fit do keras.
    
    Retorno:
    Dicionário contendo o melhor loss de treino e de validação baseado 
    no menor loss de validação.
    """
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    idx_min_val_loss = np.argmin(val_loss)
    return {'train_loss': train_loss[idx_min_val_loss], 'val_loss': val_loss[idx_min_val_loss]}


In [ ]:
def plot_training_error_curves(history):
    """Função para plotar as curvas de erro do treinamento da rede neural.
    
    Argumento(s):
    history -- Objeto retornado pela função fit do keras.
    
    Retorno:
    A função gera o gráfico do treino da rede e retorna None.
    """
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    fig, ax = plt.subplots()
    ax.plot(train_loss, label='Train')
    ax.plot(val_loss, label='Validation')
    ax.set(title='Training and Validation Error Curves', xlabel='Epochs', ylabel='Loss (MSE)')
    ax.legend()
    plt.show()

In [ ]:
def predict(classifier, history):
    ## Fazer predições no conjunto de teste
    y_pred = classifier.predict(X_test)
    y_pred_class = classifier.predict_classes(X_test, verbose=0)

    ## Matriz de confusão
    print('Matriz de confusão')
    print(confusion_matrix(y_test, y_pred_class))

    ## Computar métricas de desempenho
    losses = extract_final_losses(history)
    print("{metric:<18}{value:.4f}".format(metric="Train Loss:", value=losses['train_loss']))
    print("{metric:<18}{value:.4f}".format(metric="Validation Loss:", value=losses['val_loss']))
    print("{metric:<18}{value:.4f}".format(metric="MSE:", value=mean_squared_error(y_test, y_pred)))
    print("{metric:<18}{value:.4f}".format(metric="Accuracy:", value=accuracy_score(y_test, y_pred_class)))
    print("{metric:<18}{value:.4f}".format(metric="Recall:", value=recall_score(y_test, y_pred_class)))
    print("{metric:<18}{value:.4f}".format(metric="Precision:", value=precision_score(y_test, y_pred_class)))
    print("{metric:<18}{value:.4f}".format(metric="F1:", value=f1_score(y_test, y_pred_class)))
    print("{metric:<18}{value:.4f}".format(metric="AUROC:", value=roc_auc_score(y_test, y_pred)))
    print("")
    return losses['train_loss'], losses['val_loss'], mean_squared_error(y_test, y_pred), accuracy_score(y_test, y_pred_class), recall_score(y_test, y_pred_class), precision_score(y_test, y_pred_class), f1_score(y_test, y_pred_class), roc_auc_score(y_test, y_pred)

In [ ]:
def run_experiments(n=10):
    mse_train = []
    mse_val = []
    mse_test = []
    accuracy = []
    recall = []
    precision = []
    f1 = []
    auroc = []
    for i in range(n):
        print("Experiment " + str(i+1))
        classifier = create_network()
        history = train_network(classifier)
        extract_final_losses(history)
        plot_training_error_curves(history)
        msetr, msevl, msets, acc, rec, prc, f1_score, roc = predict(classifier, history)
        mse_train.append(msetr)
        mse_val.append(msevl)
        mse_test.append(msets)
        accuracy.append(acc)
        recall.append(rec)
        precision.append(prc)
        f1.append(f1_score)
        auroc.append(roc)
    return mse_train, mse_val, mse_test, accuracy, recall, precision, f1, auroc

In [ ]:
mse_train, mse_val, mse_test, accuracy, recall, precision, f1, auroc = run_experiments()

In [ ]:
print("Mean")
print("MSE Train")
print(np.mean(mse_train))
print("MSE Test")
print(np.mean(mse_test))
print("MSE Validation")
print(np.mean(mse_val))
print("Accuracy")
print(np.mean(accuracy))
print("Recall")
print(np.mean(recall))
print("Precision")
print(np.mean(precision))
print("F1")
print(np.mean(f1))
print("AUROC")
print(np.mean(auroc))

print("Standart Deviation")
print("MSE Train")
print(np.std(mse_train))
print("MSE Test")
print(np.std(mse_test))
print("MSE Validation")
print(np.std(mse_val))
print("Accuracy")
print(np.std(accuracy))
print("Recall")
print(np.std(recall))
print("Precision")
print(np.std(precision))
print("F1")
print(np.std(f1))
print("AUROC")
print(np.std(auroc))
print("")

In [ ]:
print(learning_rate)
print(hidden_layers)
print(act_function)
print("")
print("{s1},{s2},{s3},{s4},{s5},{s6},{s7},{s8},{s9},{s10},{s11},{s12},{s13},{s14},{s15},{s16}"
      .format(s1=np.mean(mse_train), s2=np.mean(mse_test), s3=np.mean(mse_val), s4=np.mean(accuracy),
             s5=np.mean(recall), s6=np.mean(precision), s7=np.mean(f1), s8=np.mean(auroc),
             s9=np.std(mse_train), s10=np.std(mse_test), s11=np.std(mse_val), s12=np.std(accuracy),
             s13=np.std(recall), s14=np.std(precision), s15=np.std(f1), s16=np.std(auroc)))
print("")